In [14]:
%pip install -r data-requirements.txt

  Using cached absl_py-1.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached anyio-4.2.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached argon2_cffi-23.1.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached argon2_cffi_bindings-21.2.0-cp38-abi3-macosx_10_9_universal2.whl.metadata (6.7 kB)
  Using cached arrow-1.3.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached asttokens-2.0.5-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached async_lru-2.0.4-py3-none-any.whl.metadata (4.5 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached Babel-2.14.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached backcall-0.2.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached bleach-6.1.0-py3-none-any.whl.metadata (30 kB)
  Using cached cachetools-5.2.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2022.6.15-py3-none-

In [15]:
import json
import os
import typing as t

# data prep
import pandas as pd
import numpy as np
import pyodbc

# for creating image vector embeddings
from PIL import Image
from img2vec_pytorch import Img2Vec

# for creating semantic (text-based) vector embeddings
from sentence_transformers import SentenceTransformer

# for Redis
import redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query

from dotenv import load_dotenv

# load connection info from .env
load_dotenv('../.env')
DB_SERVER=os.environ.get('DB_SERVER')
DB_NAME=os.environ.get('DB_NAME')
DB_USERNAME=os.environ.get('DB_USERNAME')
DB_PASSWORD=os.environ.get('DB_PASSWORD')
DB_LIMIT=int(os.environ.get('DB_LIMIT'))


In [16]:
# connect to database, load in data
connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={DB_SERVER};DATABASE={DB_NAME};UID={DB_USERNAME};PWD={DB_PASSWORD};Encrypt=yes;TrustServerCertificate=yes'
conn = pyodbc.connect(connection_string) 

product_query = f'SELECT TOP {DB_LIMIT} [id],[gender],[masterCategory],[subCategory],[articleType],[baseColour],[season],[year],[usage],[productDisplayName] FROM [aidemo].[styles]'

df = pd.read_sql_query(product_query, conn)

# Display the DataFrame
df.info()

# Close the connection
conn.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1000 non-null   int64 
 1   gender              1000 non-null   object
 2   masterCategory      1000 non-null   object
 3   subCategory         1000 non-null   object
 4   articleType         1000 non-null   object
 5   baseColour          1000 non-null   object
 6   season              1000 non-null   object
 7   year                1000 non-null   int64 
 8   usage               1000 non-null   object
 9   productDisplayName  1000 non-null   object
dtypes: int64(2), object(8)
memory usage: 78.3+ KB


/var/folders/5_/5r7_byn103q6wd4ht6bng8lh0000gp/T/ipykernel_88508/987698843.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(product_query, conn)


In [17]:
df["product_text"] = df.apply(lambda row: f"name {row['productDisplayName']} category {row['masterCategory']} subcategory {row['subCategory']} color {row['baseColour']} gender {row['gender']}".lower(), axis=1)
df.rename({"id":"product_id"}, inplace=True, axis=1)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   product_id          1000 non-null   int64 
 1   gender              1000 non-null   object
 2   masterCategory      1000 non-null   object
 3   subCategory         1000 non-null   object
 4   articleType         1000 non-null   object
 5   baseColour          1000 non-null   object
 6   season              1000 non-null   object
 7   year                1000 non-null   int64 
 8   usage               1000 non-null   object
 9   productDisplayName  1000 non-null   object
 10  product_text        1000 non-null   object
dtypes: int64(2), object(9)
memory usage: 86.1+ KB


In [18]:
# check out one of the texts we will use to create semantic embeddings
df["product_text"][0]

'name nike sahara team india fanwear round neck jersey category apparel subcategory topwear color blue gender men'

In [19]:
# Resnet-18 to create image embeddings
img2vec = Img2Vec(cuda=False)

# bert variant to create text embeddings
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

/Users/roberto.perez/Documents/Projects/redis-azure-ai-demo/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/roberto.perez/Documents/Projects/redis-azure-ai-demo/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
def get_batch(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def generate_image_vectors(products, image_base_path, batch_size=1000):
    output_dict={}

    for batch in get_batch(products, batch_size):
        product_ids = batch['product_id'].values.tolist()
        image_filenames = [image_base_path + "/" + str(_id) + ".jpg" for _id in product_ids]
        images=[]
        converted=[]

        for img_path, _id in zip(image_filenames, product_ids):
            try:
                img = Image.open(img_path).convert('RGB')
                img = img.resize((224, 224))
                images.append(img)
                converted.append(_id)
            except:
                #unable_to_convert -> skip to the next image
                continue

        #Generate vectors for all images in this batch
        vec_list = img2vec.get_vec(images)

        #update the dictionary to be returned
        batch_dict= dict(zip(converted, vec_list))
        output_dict.update(batch_dict)
        print(f"Processed {str(batch_size)} product images")

    return output_dict

def generate_text_vectors(products_df):
    text_vectors = {}
    # generate text vector
    for index, row in products_df.iterrows():
        text_vector = model.encode(row["product_text"])
        text_vectors[row["product_id"]] = text_vector.astype(np.float32)
    
    print(f"Processed {str(len(text_vectors))} product text fields")
    return text_vectors

# combine into a single json file
def combine_vector_dicts(txt_vectors, img_vectors, products):
    product_vectors = []
    for _, row in products.iterrows():
        try:
            _id = row["product_id"]
            text_vector = txt_vectors[_id].tolist()
            img_vector = img_vectors[_id].tolist()
            vector_dict = {
                "text_vector": text_vector,
                "img_vector": img_vector,
                "product_id": _id
            }
            product_vectors.append(vector_dict)
        except KeyError:
            continue
    return product_vectors

def write_product_vector_json(vector_dict):
    product_vector_json = json.dumps(vector_dict)
    with open("./product_vectors.json", "w") as f:
        f.write(product_vector_json)

def write_product_metadata_json(metadata):

    products_json = json.dumps(metadata)
    with open("./product_metadata.json", "w") as f:
        f.write(products_json)

def create_product_metadata(metadata_df, image_base_path):
    products = []
    for _, row in metadata_df.iterrows():
        product = {
            "product_id": row["product_id"],
            # create a text based representation to create a semantic embedding with
            "product_metadata": {
                "name": row["productDisplayName"],
                "gender": row["gender"],
                "master_category": row["masterCategory"],
                "sub_category": row["subCategory"],
                "article_type": row["articleType"],
                "base_color": row["baseColour"],
                "season": row["season"],
                "year": row["year"],
                "usage": row["usage"],
                "image_url": image_base_path + "/" + str(row["product_id"]) + ".jpg",
                "keywords": '',
                "brand": '',
                "age_group": ''
            }
        }
        products.append(product)

    return products


In [ ]:
#process vector and metadata for products
data_path = "../app/vecsim_app/static/images"
images_base_path = "/images"

image_vectors = generate_image_vectors(df[:DB_LIMIT], data_path, DB_LIMIT)
text_vectors = generate_text_vectors(df[:DB_LIMIT])
vector_dict = combine_vector_dicts(text_vectors, image_vectors, df)
image_dim = [len(i) for i in image_vectors.values()][0]
text_dim = [len(i) for i in text_vectors.values()][0]

metadata = create_product_metadata(df[:DB_LIMIT], images_base_path)
#optional write to file system
write_product_metadata_json(metadata)
write_product_vector_json(vector_dict)